In [ ]:
import numpy as np
import cv2

image = cv2.imread('Desktop/Udemy - Computer Vision Bootcamp with Python (OpenCV) - YOLO, SSD 2020-11/19. COURSE MATERIALS (DOWNLOADS)/1.1 computer_vision_code/car_image.jpg')
original_width , original_height = image.shape[1], image.shape[0]

image.shape

# the whole model is trained in the coco dataset
# there are 80 (90) possible output classes [classes where increase from 80 to 90]
# 0:person - 2:car - 5:bus
classes = ['car','person', 'bus']

# darknet : Open source neural network
neural_network = cv2.dnn.readNetFromDarknet('Downloads/yolov3.cfg','Downloads/yolov3.weights')

In [ ]:
# define whether we run the algorithm with CPU or with GPU
# We are going to use CPU !!
neural_network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
neural_network.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
# Firstly we need to change the image into BLOB (Binary large object) [0-1]
# We represent images or data related file in blob format
# Open cv uses bgr so we use TRUE to convert RGB in BGR  

blob = cv2.dnn.blobFromImage(image, 1/255,(320,320), True)

neural_network.setInput(blob)


layer_names = neural_network.getLayerNames()
print(layer_names)

In [ ]:
# Yolo network has 3 output layer
print(neural_network.getUnconnectedOutLayers()) # this will give indixes of the output layer

In [ ]:
output_names = [layer_names[index[0]-1] for index in neural_network.getUnconnectedOutLayers()]
print(output_names)

In [ ]:
print(type(output_names))

In [ ]:
outputs = neural_network.forward(output_names)

# there are 300 predicted bounding boxes - 85 is the prediction vector
# first 5 parameters are (x,y,w,h,conf) + 80 output classes in coco dataset
print(outputs[0].shape)

In [ ]:
print(outputs[1].shape)

In [ ]:
print(outputs[2].shape)

In [ ]:
YOLO_IMAGE_SIZE = 320

def find_objects(model_outputs):
    bounding_box_locations = []
    class_ids = []
    confidence_values = []
    
    for output in model_outputs:
        for prediction in output:

            class_probabilities = prediction[5:]
            class_id = np.argmax(class_probabilities)
            # confidence is the probability of detected object
            confidence = class_probabilities[class_id]
        
            THRESHOLD = 0.3
            # WE ARE NOT GOING TO BOTHER WITH OBJECTS LESS THAN 30% PROBABILITY
        
            if confidence > THRESHOLD:
            
                # x,y,w,h,conf
                # we multiply with image size because they are in between (0-1) to get pixel value
                w,h = int(prediction[2]*YOLO_IMAGE_SIZE), int(prediction[3]*YOLO_IMAGE_SIZE)
                x,y = int(prediction[0]*YOLO_IMAGE_SIZE), int(prediction[1]*YOLO_IMAGE_SIZE)
                bounding_box_locations.append([x,y,w,h])
                class_ids.append(class_id)
                confidence_values.append(float(confidence))
        
    SUPPRESSION_THRESHOLD = 0.3
    # the lower the value: the fewer the bounding boxes will remain
    box_indexes_to_keep = cv2.dnn.NMSBoxes(bounding_box_locations, confidence_values, THRESHOLD, SUPPRESSION_THRESHOLD)
    
    return box_indexes_to_keep, bounding_box_locations, class_ids, confidence_values

def show_detected_images(img, bounding_box_ids, all_bounding_boxes, class_ids, confidence_values, width_ratio, height_ratio):
    
    for index in bounding_box_ids:
        bounding_box = all_bounding_boxes[index[0]]
        x,y,w,h = int(bounding_box[0]), int(bounding_box[1]), int(bounding_box[2]), int(bounding_box[3])
        # we have to transform the location adn coordinates because the resized image
        x = int(x*width_ratio)
        y = int(y*height_ratio)
        w = int(w*width_ratio)
        h = int(h*height_ratio)
        
        # we are not going to detect every objects just PERSON and CAR
        # openCV deals with BGR blue green red (255,0,0) then it is the blue color
        if class_ids[index[0]] == 2:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
            class_with_confidence = 'CAR' + str(int(confidence_values[index[0]] *100)) + '%'
            cv2.putText(img, class_with_confidence, (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (255,0,0),1)
        
        if class_ids[index[0]] == 0:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
            class_with_confidence = 'PERSON' + str(int(confidence_values[index[0]] *100)) + '%'
            cv2.putText(img, class_with_confidence, (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.5, (255,0,0),1)
        
        

In [ ]:
predicted_objects, bbox_locations, class_label_ids, conf_values = find_objects(outputs)

show_detected_images(image, predicted_objects, bbox_locations, class_label_ids, conf_values, 
                     original_width/YOLO_IMAGE_SIZE, original_height/YOLO_IMAGE_SIZE)

cv2.imshow('YOLO Algorithm', image)
cv2.waitKey()